# Ranking & Decision Layer
How to turn a candidate set into a final ranked list under product constraints.

This notebook shows:
- feature-level fusion (behavior score, content score, popularity, user signals)
- a simple ranker (logistic model) + cold-start gating
- post-ranking constraints: dedupe, caps, sanity metrics


## 0) Setup

In [3]:
import numpy as np
from numpy.random import default_rng
import math

rng = default_rng(22)

def l2_normalize(x, axis=1, eps=1e-12):
    n = np.linalg.norm(x, axis=axis, keepdims=True)
    return x / np.maximum(n, eps)

def ndcg_at_k(items, pos_item, k=10):
    top = items[:k].tolist()
    if int(pos_item) in top:
        r = top.index(int(pos_item)) + 1
        return 1.0 / math.log2(r + 1)
    return 0.0


## 1) Environment + fused candidate set

In [5]:
n_users = 6000
n_items = 2500
d_true = 32
d_beh = 48
d_cont = 64

U_true = rng.normal(size=(n_users, d_true)).astype(np.float32)
V_true = rng.normal(size=(n_items, d_true)).astype(np.float32)

pop = rng.power(a=2.1, size=n_items).astype(np.float32)
pop = pop / pop.sum()
pop_rank = np.argsort(-pop)

def sigmoid(x): return 1.0 / (1.0 + np.exp(-x))

def click_prob(u, i):
    a = float(U_true[u] @ V_true[i])
    a = max(min(a, 8.0), -8.0)
    p = sigmoid(a) * 0.8 + 0.2 * float(pop[i] * n_items) / 6.0
    return float(min(max(p, 0.001), 0.999))

user_emb = l2_normalize((U_true @ rng.normal(size=(d_true, d_beh)).astype(np.float32) + 0.7 * rng.normal(size=(n_users, d_beh)).astype(np.float32)).astype(np.float32))
item_emb_behavior = l2_normalize((V_true @ rng.normal(size=(d_true, d_beh)).astype(np.float32) + 0.7 * rng.normal(size=(n_items, d_beh)).astype(np.float32)).astype(np.float32))

item_emb_content = l2_normalize((V_true @ rng.normal(size=(d_true, d_cont)).astype(np.float32) + 0.8 * rng.normal(size=(n_items, d_cont)).astype(np.float32)).astype(np.float32))
user_content_q = l2_normalize((U_true @ rng.normal(size=(d_true, d_cont)).astype(np.float32) + 0.8 * rng.normal(size=(n_users, d_cont)).astype(np.float32)).astype(np.float32))

eval_users = np.arange(4500, 5800, dtype=np.int32)

def sample_true_positive(u):
    impr = rng.choice(n_items, size=600, replace=False, p=pop).astype(np.int32)
    probs = np.array([click_prob(int(u), int(i)) for i in impr], dtype=np.float64)
    probs /= probs.sum()
    return int(rng.choice(impr, p=probs))

pos_targets = np.array([sample_true_positive(int(u)) for u in eval_users], dtype=np.int32)

S_beh = (user_emb[eval_users] @ item_emb_behavior.T).astype(np.float32)
S_cont = (user_content_q[eval_users] @ item_emb_content.T).astype(np.float32)

def topk_idx(scores_row, k):
    idx = np.argpartition(-scores_row, kth=k-1)[:k]
    return idx[np.argsort(-scores_row[idx])]

def fuse_candidates(i, K_final=500, q_beh=300, q_cont=150, q_pop=50):
    beh = topk_idx(S_beh[i], q_beh)
    cont = topk_idx(S_cont[i], q_cont)
    popc = pop_rank[:q_pop]
    seen, fused = set(), []

    # First pass: try to get K_final from beh, cont, popc in order
    for arr in (beh, cont, popc):
        for x in arr:
            x = int(x)
            if x not in seen:
                fused.append(x)
                seen.add(x)
                if len(fused) >= K_final:
                    return np.array(fused, dtype=np.int32)

    # Second pass: if not enough, get more from beh
    for x in beh:
        x = int(x)
        if x not in seen:
            fused.append(x)
            seen.add(x)
            if len(fused) >= K_final:
                return np.array(fused, dtype=np.int32)

    # If still not K_final items, fill the remaining spots from popular items
    # ensuring they are not already in fused.
    if len(fused) < K_final:
        for x in pop_rank:
            x = int(x)
            if x not in seen:
                fused.append(x)
                seen.add(x)
                if len(fused) >= K_final:
                    break # Break from this loop once K_final is reached

    return np.array(fused, dtype=np.int32)

K_final = 500
cands = np.stack([fuse_candidates(i, K_final=K_final) for i in range(len(eval_users))], axis=0)

cand_recall = np.mean([1.0 if int(pos_targets[i]) in set(cands[i].tolist()) else 0.0 for i in range(len(eval_users))])
print("candidate recall:", round(cand_recall, 4), "cands:", cands.shape)


candidate recall: 0.2169 cands: (1300, 500)


## 2) Feature-level fusion + ranker training

In [6]:
hist_len = rng.integers(0, 60, size=n_users).astype(np.int32)

def build_rank_dataset(u_ids, cands, pos_items, n_negs=60, seed=1):
    rng2 = default_rng(seed)
    X, y = [], []
    for row, u in enumerate(u_ids):
        u = int(u)
        pos = int(pos_items[row])
        cand = cands[row]
        if pos not in set(cand.tolist()):
            cand = cand.copy(); cand[0] = pos
        neg_pool = cand[cand != pos]
        negs = rng2.choice(neg_pool, size=min(n_negs, len(neg_pool)), replace=False)

        def featurize(i):
            i = int(i)
            f_beh = float(user_emb[u] @ item_emb_behavior[i])
            f_cont = float(user_content_q[u] @ item_emb_content[i])
            f_pop = float(pop[i])
            f_hist = float(hist_len[u])
            return np.array([f_beh, f_cont, f_pop, f_hist], dtype=np.float32)

        X.append(featurize(pos)); y.append(1.0)
        for j in negs:
            X.append(featurize(int(j))); y.append(0.0)

    return np.stack(X), np.array(y, dtype=np.float32)

X, y = build_rank_dataset(eval_users, cands, pos_targets, n_negs=60, seed=1)
print("rank dataset:", X.shape, "pos rate:", round(y.mean(), 4))

def train_logreg(X, y, lr=0.15, epochs=6, l2=3e-4, seed=0):
    rng2 = default_rng(seed)
    n, m = X.shape
    w = np.zeros((m,), dtype=np.float32)
    b = 0.0
    def sig(z): return 1.0 / (1.0 + np.exp(-z))
    idx = np.arange(n)
    for ep in range(epochs):
        rng2.shuffle(idx)
        loss = 0.0
        for t in idx:
            x = X[t]; yt = float(y[t])
            z = float(x @ w + b)
            p = sig(z)
            loss += -(yt*math.log(max(p,1e-10)) + (1-yt)*math.log(max(1-p,1e-10)))
            g = (p - yt)
            w -= lr * (g * x + l2 * w)
            b -= lr * g
        print(f"epoch {ep+1}/{epochs} loss≈{loss/n:.4f}")
    return w, float(b)

w, b = train_logreg(X, y)
print("w:", np.round(w,3), "b:", round(b,3))


rank dataset: (79300, 4) pos rate: 0.0164
epoch 1/6 loss≈0.6248
epoch 2/6 loss≈0.6123
epoch 3/6 loss≈0.6202
epoch 4/6 loss≈0.6301
epoch 5/6 loss≈0.6263
epoch 6/6 loss≈0.6198
w: [-6.578 -3.313  0.    -1.926] b: -3.107


## 3) Cold-start gating

In [7]:
def rank_with_gating(u_ids, cands, w, b, topn=10, cold_thresh=10, boost=0.35):
    ranked = []
    for row, u in enumerate(u_ids):
        u = int(u)
        cand = cands[row]
        beh = (item_emb_behavior[cand] @ user_emb[u]).astype(np.float32)
        cont = (item_emb_content[cand] @ user_content_q[u]).astype(np.float32)
        popf = pop[cand].astype(np.float32)
        hlen = float(hist_len[u])
        if hist_len[u] < cold_thresh:
            cont = cont * (1.0 + boost)
        Xc = np.stack([beh, cont, popf, np.full_like(popf, hlen)], axis=1).astype(np.float32)
        s = (Xc @ w + b).astype(np.float32)
        idx = np.argpartition(-s, kth=topn-1)[:topn]
        idx = idx[np.argsort(-s[idx])]
        ranked.append(cand[idx])
    return np.array(ranked, dtype=np.int32)

def metrics(top_items, pos_items):
    rec = np.mean([1.0 if int(pos_items[i]) in set(top_items[i].tolist()) else 0.0 for i in range(len(pos_items))])
    nd = np.mean([ndcg_at_k(top_items[i], pos_items[i], k=top_items.shape[1]) for i in range(len(pos_items))])
    return rec, nd

N = 10
top_no_gate = rank_with_gating(eval_users, cands, w, b, topn=N, cold_thresh=-1, boost=0.0)
top_gate = rank_with_gating(eval_users, cands, w, b, topn=N, cold_thresh=10, boost=0.35)

r1, n1 = metrics(top_no_gate, pos_targets)
r2, n2 = metrics(top_gate, pos_targets)

print(f"Final top-{N} no-gate  Recall@{N}={r1:.4f}  NDCG@{N}={n1:.4f}")
print(f"Final top-{N} gated    Recall@{N}={r2:.4f}  NDCG@{N}={n2:.4f}")


Final top-10 no-gate  Recall@10=0.0038  NDCG@10=0.0022
Final top-10 gated    Recall@10=0.0038  NDCG@10=0.0018


## 4) Post-ranking constraints

In [8]:
n_clusters = 60
cluster = rng.integers(0, n_clusters, size=n_items).astype(np.int32)

def apply_cluster_cap(top_items, cap=2):
    out = []
    for row in range(top_items.shape[0]):
        seen = {}
        keep = []
        for it in top_items[row]:
            c = int(cluster[int(it)])
            if seen.get(c, 0) < cap:
                keep.append(int(it))
                seen[c] = seen.get(c, 0) + 1
        if len(keep) < top_items.shape[1]:
            for it in top_items[row]:
                if int(it) not in keep:
                    keep.append(int(it))
                if len(keep) == top_items.shape[1]:
                    break
        out.append(keep)
    return np.array(out, dtype=np.int32)

top_gate_capped = apply_cluster_cap(top_gate, cap=2)
r3, n3 = metrics(top_gate_capped, pos_targets)
print(f"Final top-{N} gated+capped Recall@{N}={r3:.4f}  NDCG@{N}={n3:.4f}")


Final top-10 gated+capped Recall@10=0.0038  NDCG@10=0.0018


## Production notes
- Ranking is the main fusion layer: combine behavior, content, popularity, and user signals.
- Gating handles cold-start with simple, testable rules.
- Post-ranking constraints enforce diversity/safety without retraining models.
- Monitor: candidate coverage, segment metrics, top-item concentration, tail latency.
